In [1]:
from env import load_uavs, load_tasks, initialize_targets
from runEnv import UAVEnv

In [2]:
# 设置数据目录
uav_csv = "data/uav.csv"
task_csv = "data/task.csv"
uavs = load_uavs(uav_csv)
tasks = load_tasks(task_csv)
targets = initialize_targets(tasks)

# 俩个模式：DQN和Attention
dqn_mode = "dqn"
attention_mode = "attention"
env = UAVEnv(uavs, targets, tasks, mode=dqn_mode)

In [3]:
"""
运行 DQN 优化
"""
# from train.dqn_train import train_dqn
# model = train_dqn(env)

'\n运行 DQN 优化\n'

In [4]:
"""
运行 myDQN 优化
"""
# from train.dqn_train import train_dqn
# model = train_dqn(env, model_name="my_dqn_model.pth")

'\n运行 myDQN 优化\n'

In [5]:
"""
运行 AttentionDQN 优化
"""
# import torch
# # 全局设置（推荐）
# torch.set_printoptions(
#     threshold=float('inf'),  # 禁用省略，显示所有元素
#     edgeitems=5,             # 触发省略时首尾显示5个元素（仅当threshold未设为inf时有效）
#     precision=6,             # 浮点数保留6位小数
#     linewidth=200            # 每行最大宽度200字符（避免换行过多）
# )

# from train.attention_train import train_attention_dqn
# model = train_attention_dqn(env)

'\n运行 AttentionDQN 优化\n'

In [6]:
"""
运行 PSO 优化
"""
# from model.pso import PSO
# pso = PSO(env, num_particles=30, max_iter=100)
# best_position, best_fitness = pso.optimize()

'\n运行 PSO 优化\n'

In [7]:
"""
运行 HS_MOPSO 优化
"""
from model.hs_mopso import HS_MOPSO
mopso = HS_MOPSO(env, max_iter=80, pop_size=120)
mopso.run()

pareto set: [<model.hs_mopso.Particle object at 0x00000221AEFE84A0>]
Iter 1, Average Reward: 0.81, Total Distance: 1383.29, Total Time: 752.38, Success Rate: 0.83
pareto set: [<model.hs_mopso.Particle object at 0x00000221AEFE84A0>, <model.hs_mopso.Particle object at 0x00000221AEFE9520>]
Iter 2, Average Reward: 0.81, Total Distance: 1383.29, Total Time: 752.38, Success Rate: 0.83
pareto set: [<model.hs_mopso.Particle object at 0x00000221AEFE84A0>, <model.hs_mopso.Particle object at 0x00000221AEFE9520>, <model.hs_mopso.Particle object at 0x00000221AEFE8E00>]
Iter 3, Average Reward: 0.81, Total Distance: 1383.29, Total Time: 752.38, Success Rate: 0.83
pareto set: [<model.hs_mopso.Particle object at 0x00000221AEFE84A0>, <model.hs_mopso.Particle object at 0x00000221AEFE9520>, <model.hs_mopso.Particle object at 0x00000221AEFE8E00>, <model.hs_mopso.Particle object at 0x00000221AEFEB8C0>]
Iter 4, Average Reward: 0.81, Total Distance: 1383.29, Total Time: 752.38, Success Rate: 0.83
pareto set: 

In [8]:
"""
运行 RR 轮询算法
"""
# from model.rr import round_robin_allocation
# round_robin_allocation(env)

'\n运行 RR 轮询算法\n'

In [9]:
"""
运行随机算法
"""
# from model.random import random_allocation
# random_allocation(env)

'\n运行随机算法\n'